In [31]:
import requests
import pandas as pd
import itertools
from datetime import datetime, timedelta

In [32]:
# Replace with your IEX Cloud API token
api_token = 'pk_b23ba153ee2045aeb5ca77fe63ab5cb0'

# Define the base URL for the IEX Cloud API
base_url = 'https://cloud.iexapis.com/stable'

# Define the symbol for SPY
symbol = 'SPY'

# Calculate yesterday's date
yesterday_date = (datetime.now() - timedelta(days=1)).strftime('%Y%m%d')

# Define the historical endpoint
endpoint = f'/stock/{symbol}/chart/date/{yesterday_date}'

# Set parameters
params = {
    'token': api_token,
}

# Make the API request
response = requests.get(f'{base_url}{endpoint}', params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Check if data is not empty
    if data:
        # Convert the data to a DataFrame
        df = pd.DataFrame(data)
    else:
        print("No data available for yesterday.")
else:
    print(f"Error: {response.status_code}, {response.text}")

In [33]:
# Data preparation
df=df[['minute','open','close']].dropna(subset=['open', 'close'])

In [34]:
# Re-indexing
df.index = range(1, len(df) + 1)

In [35]:
# Limiting data sample
df=df.head(100)

In [36]:
# Define a function to calculate profit for a single buy trade
def calculate_trade_profit(open_price, close_price):
    return close_price - open_price

# Define a function to calculate total profit for a strategy consisting of multiple buy trades
def calculate_strategy_profit(strategy, prices):
    total_profit = 0
    for buy_trade in strategy:
        open_index, close_index = buy_trade
        if open_index < close_index and close_index < len(prices):
            open_price = prices[open_index]
            close_price = prices[close_index]
            total_profit += calculate_trade_profit(open_price, close_price)
    return total_profit

# Define the prices for each row (replace with actual prices)
prices = df['open'].tolist()

# Define the possible buy trades
num_rows = 100
possible_buy_trades = [(open_index, close_index) for open_index in range(1, num_rows) for close_index in range(open_index + 1, num_rows + 1)]

# Initialize variables to store the best strategy and its profit
best_strategy = None
best_profit = float('-inf')

# Iterate through all combinations of buy trades
for strategy in itertools.combinations(possible_buy_trades, len(possible_buy_trades)):
    # Calculate profit for the current strategy
    strategy_profit = calculate_strategy_profit(strategy, prices)
    
    # Update the best strategy if the current strategy has higher profit
    if strategy_profit > best_profit:
        best_profit = strategy_profit
        best_strategy = strategy

# Print the best strategy and its profit
print("Best Strategy:", best_strategy[:10])
print("Profit:", best_profit)

Best Strategy: ((1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11))
Profit: 4150.129999999986
